In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "4"
import yaml
import numpy as np
import open3d as o3d
import cv2
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
fast_calib_path = os.path.expanduser("~/data-fast/lxc-fast-livo2-02/storage/1001-Better-02/data.bag-calibration") 
config_path = os.path.join(fast_calib_path, "config.yaml")

pcd_and_image_override = None
config_path

In [ ]:
with open(config_path, "r") as f:
    config = yaml.safe_load(f)

def path_fix(path):
    return path
    # return path.replace("$(find fast_calib)", fast_calib_path).replace("/home/ubuntu/", os.path.expanduser("~/data-fast/lxc-fast-livo2-02/"))

config["output_path"] = path_fix(config["output_path"])
output_path = config["output_path"]
image_path = path_fix(config["image_path"])
pcd_path = os.path.join(output_path, "cloud_input.pcd")
print(config)

if pcd_and_image_override is not None:
    with open(pcd_and_image_override, "r") as f:
        pcd_and_image_from_config = yaml.safe_load(f)
    image_path = path_fix(pcd_and_image_from_config["image_path"])
    pcd_path = os.path.join(path_fix(pcd_and_image_from_config["output_path"]), "cloud_input.pcd")

calib_image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
K = np.asarray([
    [config["fx"], 0, config["cx"]],
    [0, config["fy"], config["cy"]],
    [0, 0, 1],
])
D = np.asarray([config["k1"], config["k2"], config["p1"], config["p2"]])
calib_image_raw = calib_image
calib_image = cv2.undistort(calib_image, K, D, None, K)
K, D

In [ ]:
with open(os.path.expanduser(os.path.join(output_path, "calib_result.txt")), "r") as f:
    calib = yaml.safe_load(f)
calib

In [ ]:
pcd = o3d.t.io.read_point_cloud(pcd_path)

In [ ]:
lidar2camera = np.eye(4)
lidar2camera[:3, :3] = np.asarray(calib["Rcl"]).reshape((3, 3))
lidar2camera[:3, -1] = np.asarray(calib["Pcl"])

lidar2camera

In [ ]:
plt.imshow(calib_image)

In [ ]:
xyzs = pcd.point.positions.cpu().numpy()
intensities = pcd.point.intensity.cpu().numpy()
# normals = pcd.point.normals.cpu().numpy()

In [ ]:
projected = np.zeros((calib_image.shape[0], calib_image.shape[1]))
xyzs_in_camera = xyzs @ lidar2camera[:3, :3].T + lidar2camera[:3, -1]
xyzs_in_pixels = xyzs_in_camera @ K.T
xyzs_in_image = xyzs_in_pixels / xyzs_in_pixels[:, -1:]

In [ ]:
visible_point_mask = np.logical_and(
    xyzs_in_camera[:, -1] > 0,
    np.logical_and(
        np.prod(xyzs_in_image[:, :2] > 0, axis=-1).astype(bool),
        np.prod(np.round(xyzs_in_image[:, :2]) < np.asarray(projected.shape[::-1]), axis=-1).astype(bool),
    ),
)
visible_point_mask.sum(), xyzs.shape[0]

In [ ]:
visible_point_xys = np.round(xyzs_in_image[visible_point_mask, :2]).astype(np.int32)
visible_point_colors = intensities[visible_point_mask, 0]
visible_point_colors = (visible_point_colors - visible_point_colors.min()) / visible_point_colors.mean()
visible_point_colors = np.clip(visible_point_colors, a_min=0., a_max=1.)
# visible_point_colors = np.power(visible_point_colors + 1e-4, 2.)

In [ ]:
projected = np.zeros((calib_image.shape[0], calib_image.shape[1]))
image_width = projected.shape[1]
image_heigh = projected.shape[0]
for offset in [(0, 0), (-1, 0), (1, 0), (0, 1), (0, -1)]:
# for offset in [(0, 0), (-1, 0)]:
    projected[np.clip(visible_point_xys[:, 1] + offset[1], a_min=0, a_max=image_heigh - 1), np.clip(visible_point_xys[:, 0] + offset[0], a_min=0, a_max=image_width - 1)] = visible_point_colors

In [ ]:
colored_projection = (np.asarray(matplotlib.colormaps["cividis"].colors)[(projected * 255).astype(np.int32)] * 255).astype(np.uint8)
fused_colored_projection = (0.7 * colored_projection + 0.3 * calib_image).astype(np.uint8)
fig, ax = plt.subplots()
fig.set_dpi(600)
ax.axis("off")
ax.imshow(np.concatenate([colored_projection, calib_image, fused_colored_projection], axis=1))

y_lines = range(0, colored_projection.shape[0], colored_projection.shape[0] // 32)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
for idx, y in enumerate(y_lines):
    ax.hlines(y, xmin=0, xmax=colored_projection.shape[1] * 3 - 1, color=colors[idx % len(colors)], linewidth=0.1)
plt.savefig(os.path.join(os.path.dirname(config_path), "projection-horizontal.png"), dpi=1200, bbox_inches='tight', pad_inches=0)
# plt.show()

In [ ]:
colored_projection = (np.asarray(matplotlib.colormaps["cividis"].colors)[(projected * 255).astype(np.int32)] * 255).astype(np.uint8)
fig, ax = plt.subplots()
fig.set_dpi(600)
ax.axis("off")
ax.imshow(np.concatenate([colored_projection, calib_image], axis=0))

x_lines = range(0, colored_projection.shape[1], colored_projection.shape[1] // 32)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
for idx, x in enumerate(x_lines):
    ax.vlines(x, ymin=0, ymax=(colored_projection.shape[0] * 2) - 1, color=colors[idx % len(colors)], linewidth=0.1)
plt.savefig(os.path.join(os.path.dirname(config_path), "projection-vertical.png"), dpi=1200, bbox_inches='tight', pad_inches=0)
# plt.show()